In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import seaborn as sns
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

## Functions
Iterate through each super bowl link (https://www.pro-football-reference.com/super-bowl) and scrape the box score. Final result is a dataFrame with 10 columns: 1 for each conference to hold the team name, and 1 for each quarter for each conference that holds the cumulative score's final digit

In [2]:
def scrapeURL(URL):
    page = requests.get(URL)
    df_list = pd.read_html(page.text) # create a list from the webpage
    df = df_list[0]
    
    # get the team names
    team1 = df["Unnamed: 1"][0]
    team2 = df["Unnamed: 1"][1]
    teams = [team1, team2]
    
    # get the scores
    scores1 = [int(df["1"][0]), int(df["2"][0]), int(df["3"][0]), int(df["4"][0])]
    scores2 = [int(df["1"][1]), int(df["2"][1]), int(df["3"][1]), int(df["4"][1])]
    scores = [scores1, scores2]
    
    return([teams, scores])

In [5]:
def getURLs():
    url = "https://www.pro-football-reference.com/super-bowl"
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    base = "https://www.pro-football-reference.com/"
    
    url_list = []
    table = soup.find(class_="table_container")  # table object -> body -> rows -> data -> link
    tbody = table.find('tbody')
    tr_body = tbody.find_all('tr')
    for trb in tr_body:
        for td in trb.find('td'):
            try: 
                link = td.get('href')
                if(link != "None"):
                    url_list.append(base + link)
            except:
                pass
    
    return(url_list)                 

In [4]:
# Step 3
def iterateSuperBowls(data):
    all_teams = []
    all_scores = []
    for row in data:
        teams, scores = scrapeURL(row)
        all_teams.append(teams)
        all_scores.append(scores)
    
    return([all_teams, all_scores])      

In [6]:
def checkConf(df):
    NFC = ["Dallas Cowboys", "Philadelphia Eagles", "New York Giants", "Washington Football Team",
          "Green Bay Packers", "Minnesota Vikings", "Chicago Bears", "Detroit Lions",
          "Tampa Bay Buccaneers", "New Orleans Saints", "Atlanta Falcons", "Carolina Panthers",
          "Arizona Cardinals", "Los Angeles Rams", "Seattle Seahawks", "San Francisco 49ers",
          "Washington Redskins", "St. Louis Rams"]
    
    for i in range(len(df["AFC"])):  
        if(df["AFC"].iloc[i] in NFC):                      
            dummy = df["AFC"].iloc[i]      # store name to be swapped and make swap
            df["AFC"].iloc[i] = df["NFC"].iloc[i]
            df["NFC"].iloc[i] = dummy
 
            temp = df["AFC scores"].iloc[i]   # store score to be swapped and make swap
            df["AFC scores"].iloc[i] = df["NFC scores"].iloc[i]
            df["NFC scores"].iloc[i] = temp           
    
    return df

In [8]:
def quarter_scores(df):
    # Create new column for individual quarters and set initial value to 0
    df["AFC Q1"] = df.apply(lambda x: 0, axis=1)
    df["AFC Q2"] = df.apply(lambda x: 0, axis=1)
    df["AFC Q3"] = df.apply(lambda x: 0, axis=1)
    df["AFC Q4"] = df.apply(lambda x: 0, axis=1)
    df["NFC Q1"] = df.apply(lambda x: 0, axis=1)
    df["NFC Q2"] = df.apply(lambda x: 0, axis=1)
    df["NFC Q3"] = df.apply(lambda x: 0, axis=1)
    df["NFC Q4"] = df.apply(lambda x: 0, axis=1)

    # update score with only last digit for each quarter
    for i in range(len(combined_df["AFC"])):
        df["AFC Q1"].iloc[i] = df["AFC scores"].iloc[i][0] % 10
        df["AFC Q2"].iloc[i] = (df["AFC scores"].iloc[i][1] + df["AFC Q1"].iloc[i]) % 10
        df["AFC Q3"].iloc[i] = (df["AFC scores"].iloc[i][2] + df["AFC Q2"].iloc[i]) % 10
        df["AFC Q4"].iloc[i] = (df["AFC scores"].iloc[i][3] + df["AFC Q3"].iloc[i]) % 10
        df["NFC Q1"].iloc[i] = df["NFC scores"].iloc[i][0] % 10
        df["NFC Q2"].iloc[i] = (df["NFC scores"].iloc[i][1] + df["NFC Q1"].iloc[i]) % 10
        df["NFC Q3"].iloc[i] = (df["NFC scores"].iloc[i][2] + df["NFC Q2"].iloc[i]) % 10
        df["NFC Q4"].iloc[i] = (df["NFC scores"].iloc[i][3] + df["NFC Q3"].iloc[i]) % 10
    
    df.drop(['AFC scores', 'NFC scores'], axis=1, inplace=True)
    
    return df

## Driver    
**getUrls** scrapes the website for all urls to individual super bowl box squares     
**iterateSuperBowls** goes through each super bowl url and calls **scrapeURL** to get the data, and then adds the data to teams and scores dataframes

The data is ordered by descending super bowl number by default, so teams and scores are reversed to get them in ascending order. **checkConf** will check for NFC teams labeled as AFC, and switch values to appropriate column if needed. 

Note: SuperBowl 3 featured the New York Jets and the Baltimore Colts, where both teams are currently in the AFC. The Jets got labeled as the AFC team and I decided to leave it

**quarter_scores** creates columns for each quarter and calculates the cumulative final digit

In [9]:
teams, scores = iterateSuperBowls(getURLs())
teams.reverse()
scores.reverse()

teams_df = pd.DataFrame(teams, columns=["AFC","NFC"])
scores_df = pd.DataFrame(scores, columns=["AFC scores", "NFC scores"])
combined_df = teams_df.join(scores_df, how="inner")

# fix conference labeling
cleaned_df = checkConf(combined_df)

# create final DataFrame
final_df = quarter_scores(cleaned_df)

## Export data

In [11]:
final_df.to_csv('data.csv', index=False) 

# include initial raw data for other analysis if wanted
teams_df.to_csv('teams.csv', index=False)
scores_df.to_csv('scores.csv', index=False)